<a href="https://colab.research.google.com/github/Sid-202/ML-Project-Customer-Goodness-Credit-/blob/main/customer_credit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Customer Credit Goodness Project


In [2]:
file_master = '/content/drive/MyDrive/ML proj/customer goodness/Customer_Credit_Management_Master.xlsx'
file_receivable = '/content/drive/MyDrive/ML proj/customer goodness/ReceivableFromCustomer_Aging.xlsx'


In [28]:
import pandas as pd
import numpy as np
import functools

Get the data

In [179]:
file1 = pd.read_excel(file_master)
file2 = pd.read_excel(file_receivable)

In [180]:
# Rename all col names with whitespace for easy access
newcols1 = {i:'_'.join(i.split(' ')) for i in file1.columns}
file1.rename(columns=newcols1, inplace=True)
newcols2 = {i:'_'.join(i.split(' ')) for i in file2.columns}
file2.rename(columns=newcols2, inplace=True)

file2.rename(columns={'Customer_Number':"Customer"}, inplace=True)

print(file1.shape)
print(file1.columns)
print(file2.shape)
print(file2.columns)

(12827, 34)
Index(['Customer', 'Credit_control_area', 'Credit_limit', 'Credit_account',
       'Sales_value', 'Total_receivables', 'Special_liabilities',
       'Exceeded_on', 'Reset', 'Created_on', 'Risk_category',
       'Last_internal_review', 'Blocked', 'Credit_rep.group',
       'Next_internal_review', 'Credit_information_number', 'D&B_indicator',
       'D&B_rating', 'Last_ext.review', 'Changed_on', 'Text_changed_on',
       'Cust.cred.group', 'Reference_date', 'Customer_group',
       'Date_of_last_pmnt', 'Amnt_of_last_payment', 'Currency',
       'Payment_index', 'Rating', 'Recmd._Credit_Limit', 'Currency.1',
       'Monitoring', 'Secured_receivables', 'Stale/Bad_Debt_Balance'],
      dtype='object')
(36052, 38)
Index(['Plant', 'Customer', 'Business_Segment', 'State', 'Collector',
       'Lease_state', 'Last_Ticket_Date', 'Document_No', 'Company_Code',
       'Year', 'Cleared_Doc', 'Ref_Doc', 'Ref.Key1', 'Ref.key2', 'Doc_Type',
       'GL_Acoount', 'Billing_Document', 'Total_Ta

Remove empty and single entry columns

In [181]:
rem1 = []
rem2 = []
ind=0
for i in file1.isnull().values.all(axis=0):
  if i == True:
    rem1.append(file1.columns[ind])
  ind+=1
ind=0
for i in file2.isnull().values.all(axis=0):
  if i == True:
    rem2.append(file2.columns[ind])
  ind+=1 
ind=0
for i in file1.apply(lambda col: len(col.unique())):
  if i == 1:
    rem1.append(file1.columns[ind])
  ind+=1
ind=0
for i in file2.apply(lambda col: len(col.unique())):
  if i == 1:
    rem2.append(file2.columns[ind])
  ind+=1
print(rem1)
print(rem2)

print(f"file1 shape before-{file1.shape}; file2 shape before-{file2.shape}")
file1.drop(columns=rem1, inplace=True)
file2.drop(columns=rem2, inplace=True)
print(f"Remaining columns:")
print(file1.columns)
print(file2.columns)
print(f"file1 shape after-{file1.shape}; file2 shape after-{file2.shape}")

['Reset', 'Last_internal_review', 'Credit_rep.group', 'Next_internal_review', 'Credit_information_number', 'D&B_rating', 'Last_ext.review', 'Reference_date', 'Currency.1', 'Monitoring', 'Credit_control_area', 'Sales_value', 'Special_liabilities', 'Reset', 'Last_internal_review', 'Credit_rep.group', 'Next_internal_review', 'Credit_information_number', 'D&B_indicator', 'D&B_rating', 'Last_ext.review', 'Reference_date', 'Recmd._Credit_Limit', 'Currency.1', 'Monitoring', 'Secured_receivables']
[]
file1 shape before-(12827, 34); file2 shape before-(36052, 38)
Remaining columns:
Index(['Customer', 'Credit_limit', 'Credit_account', 'Total_receivables',
       'Exceeded_on', 'Created_on', 'Risk_category', 'Blocked', 'Changed_on',
       'Text_changed_on', 'Cust.cred.group', 'Customer_group',
       'Date_of_last_pmnt', 'Amnt_of_last_payment', 'Currency',
       'Payment_index', 'Rating', 'Stale/Bad_Debt_Balance'],
      dtype='object')
Index(['Plant', 'Customer', 'Business_Segment', 'State', '

Check if customers in both files are same

In [183]:
if functools.reduce(lambda x, y : x and y, map(lambda p, q: p == q,file1.Customer.unique(),file2.Customer.unique()), True):
    print("The lists l1 and l2 are the same")
else:
    print("The lists l1 and l2 are not the same")


print(f"Customers - master unique-{len(file1.Customer.unique())}, dtype={file1.Customer.dtypes}; ageing unique-{len(file2.Customer.unique())}, dtype={file2.Customer.dtypes}")

# convert both customer cols to string
file2['Customer'] = file2['Customer'].astype(str)
file1['Customer'] = file1['Customer'].astype(str)

# simplify customers in file2
f1customers = file1.Customer.unique()
app = []
for val in file2.Customer:
  if val not in f1customers:
    if val[3:] in f1customers:
      val = val[3:]
  app.append(val)

file2['Customer'] = app

# Check for values not in file 1 
in2notin1 = [i for i in file2.Customer.unique() if i not in file1.Customer.unique()]
print(f"len customers only in file 2 - {len(in2notin1)}")
print(in2notin1)


The lists l1 and l2 are not the same
Customers - master unique-12827, dtype=object; ageing unique-800, dtype=object
len customers only in file 2 - 12
['0002008079', '1205WS', '2304FS', '1312WS', '1209WS', '1908CR', '2550FS', '1915CR', '8105WS', '1917CR', '8110WS', 'nan']


In [187]:
# Remove rows with customers not in main dataset
print(f"shape before-{file2.shape}")

for i in in2notin1:
  file2.drop(file2[file2.Customer == i].index, inplace = True)
file2 = file2[file2.Customer.notnull()]

print(f"Shape after- {file2.shape}")

in2notin1 = [i for i in file2.Customer.unique() if i not in file1.Customer.unique()]
print(f"len customers only in file 2 - {len(in2notin1)}")
print(in2notin1)

shape before-(35998, 38)
Shape after- (35998, 38)
len customers only in file 2 - 0
[]


Merge the two datasets

In [188]:
print(file1.shape)
print(file2.shape)


a1 = file1.copy()
a2 = file2.copy()

a = a1.merge(a2, how='outer')
print(a)
print(a.columns)


# print(file1.apply(lambda col: len(col.unique())))
# print(max(file2.apply(lambda col: len(col.unique()))))
# print(file2.apply(lambda col: len(col.unique())))

# v = file2.Billing_Document.value_counts()
# file2[file2.Billing_Document.isin(v.index[v.gt(1)])]
# print(v.gt(1).all())

(12827, 18)
(35998, 38)
      Customer  Credit_limit Credit_account  Total_receivables Exceeded_on  \
0         1508           0.0           1508                0.0         NaT   
1         1557           0.0           1557                0.0         NaT   
2         1812           0.0           1812                0.0         NaT   
3      1000000           1.0        1000000                0.0         NaT   
4      1000001           1.0        1000001                0.0         NaT   
...        ...           ...            ...                ...         ...   
48032   1927CR           0.0         1927CR                0.0         NaT   
48033   2104CR           0.0         2104CR                0.0         NaT   
48034   2109CR           0.0         2109CR                0.0         NaT   
48035   2518FS    99999999.0         2518FS                0.0         NaT   
48036   2561FS           0.0         2561FS                0.0         NaT   

      Created_on  Risk_category Blocked

Identify relevant features

In [191]:
main_file = a.copy()
print(main_file.columns)

relevant_feat = ['Customer', 'Credit_limit', 'Total_receivables', 'Risk_category', 'Blocked', 'Cust.cred.group',
                 'Currency', 'Payment_index', 'Rating', 'Stale/Bad_Debt_Balance', 'Plant',
       'Business_Segment', 'State', 'Collector', 'Lease_state', 'Last_Ticket_Date', 'Document_No', 'Company_Code', 'Year',
       'Cleared_Doc', 'Ref_Doc', 'Ref.Key1', 'Ref.key2', 'Doc_Type', 'GL_Acoount', 'Billing_Document', 'Total_Tax', 'Bill_to_party',
       'EDI/Non_EDI', 'Credit_Limit', 'D/C_indic.', 'Baseline_date', 'Document_date', 'Profit_Center', '<0-Days', '0-30days', '31-60days',
       '61-90days', '91-120days', '121-180days', '181_More', 'Over_90', 'Totalamount', 'Payment_Terms_Id', 'Region',
       'Credit_management:Risk_category', 'Age']['Customer', 'Credit_limit', 'Risk_category', 'Blocked', 'Cust.cred.group', ]
#Total_receivables, Exceeded_on, Created_on, Changed_on, 'Date_of_last_pmnt', 'Amnt_of_last_payment'
# removed - Credit_account, Text_changed_on, Customer_group, 

Index(['Customer', 'Credit_limit', 'Credit_account', 'Total_receivables',
       'Exceeded_on', 'Created_on', 'Risk_category', 'Blocked', 'Changed_on',
       'Text_changed_on', 'Cust.cred.group', 'Customer_group',
       'Date_of_last_pmnt', 'Amnt_of_last_payment', 'Currency',
       'Payment_index', 'Rating', 'Stale/Bad_Debt_Balance', 'Plant',
       'Business_Segment', 'State', 'Collector', 'Lease_state',
       'Last_Ticket_Date', 'Document_No', 'Company_Code', 'Year',
       'Cleared_Doc', 'Ref_Doc', 'Ref.Key1', 'Ref.key2', 'Doc_Type',
       'GL_Acoount', 'Billing_Document', 'Total_Tax', 'Bill_to_party',
       'EDI/Non_EDI', 'Credit_Limit', 'D/C_indic.', 'Baseline_date',
       'Document_date', 'Profit_Center', '<0-Days', '0-30days', '31-60days',
       '61-90days', '91-120days', '121-180days', '181_More', 'Over_90',
       'Totalamount', 'Payment_Terms_Id', 'Region',
       'Credit_management:Risk_category', 'Age'],
      dtype='object')


In [212]:
# Credit account col check
if functools.reduce(lambda x, y : x and y, map(lambda p, q: p == q,main_file.Customer,main_file.Credit_account), True):
  print("Columns Customer and Credit_account are same.")

# Credit_account is not necessary since it is the same as Customer

Columns Customer and Credit_account are same.
null elements - 36030; total elements - 48037


In [214]:
k=0
for i in main_file.Text_changed_on:
  if pd.isnull(i):
    k+=1
  else:
    print(i)
print(f"null elements - {k}; total elements - {len(main_file.Exceeded_on)}")

# Text_changed_on can be removed as only affects <0.001% of data

2012-12-21 00:00:00
2012-10-08 00:00:00
2012-10-08 00:00:00
2012-10-08 00:00:00
2011-05-07 00:00:00
2012-11-12 00:00:00
null elements - 48031; total elements - 48037


In [215]:
k=0
for i in main_file.Customer_group:
  if pd.isnull(i):
    k+=1
  else:
    print(i)
print(f"null elements - {k}; total elements - {len(main_file.Exceeded_on)}")

# Customer_group can be removed as only affects <0.001% of data

DNU
STALE
STALE
CAOF
CAOF
CAOF
null elements - 48031; total elements - 48037


In [216]:
d
k=0
for i in main_file.Currency:
  if pd.isnull(i):
    k+=1
  else:
    print(i)
print(f"null elements - {k}; total elements - {len(main_file.Exceeded_on)}")

# Customer_group can be removed as only affects <0.001% of data

Streaming output truncated to the last 5000 lines.
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
USD
U

In [147]:
ind = 0
for val in a.loc[:,'Customer']:
  if val != "" and val in file2.Customer_Number:
    a.at[ind][]

  ind += 1

Check if both customers are same

In [ ]:
for file1.col

In [46]:
for i in file2.loc[:,'Customer Number'].unique():
  print(i)


AttributeError: ignored

In [38]:

in1notin2 = ['000'+i for i in in1notin2]
intersect = 0
for i in in1notin2:
  if i in in2notin1:
    intersect+=1
print(intersect)


len only 1=12826; len only 2-799
0001000004
0001000017
0001000024
0001000026
0001000048
0001000062
0001000070
0001000088
0001000093
0001000094
0001000109
0001000123
0001000134
0001000156
0001000158
0001000190
0001000191
0001000197
0001000206
0001000263
0001000272
0001000274
0001000283
0001000287
0001000295
0001000341
0001000355
0001000372
0001000419
0001000432
0001000448
0001000458
0001000472
0001000491
0001000494
0001000510
0001000527
0001000535
0001000537
0001000544
0001000545
0001000547
0001000550
0001000558
0001000567
0001000568
0001000583
0001000606
0001000630
0001000648
0001000659
0001000677
0001000697
0001000701
0001000704
0001000721
0001000731
0001000732
0001000745
0001000764
0001000773
0001000775
0001000789
0001000796
0001000797
0001000804
0001000808
0001000827
0001000831
0001000840
0001000848
0001000885
0001000896
0001000908
0001000910
0001000940
0001000956
0001000963
0001000968
0001000979
0001000980
0001001028
0001001033
0001001034
0001001052
0001001063
0001001140
0001001152